In [1]:
# 네이버 바이브 -> 왼쪽 차트 -> 오늘의 Top 100 -> 노래 제목, 아티스트 뽑을 것

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
# 우클릭 검사 -> 왼쪽위 마우스모양 클릭 -> 찾을 부분 클릭 -> <a> 안에 class=link_text로 제목 있음

In [3]:
url = 'https://vibe.naver.com/chart/total'
html_music = requests.get(url).text
soup_music = BeautifulSoup(html_music, 'lxml')

In [4]:
titles = soup_music.select('span.inner_cell a.link_text')
print(len(titles))  # len이 0이 나옴 -> 뭔가 잘못됨

0


In [5]:
print(soup_music)  # 확인해보니 구글 검사로 본거랑 좀 다름 -> 우리가 원하던게 아님
# url을 확인했지만 이상은 없어보임 -> microsoft edge로 확인하면 팝업이 뜸. -> 팝업을 죽여야 원하는게 뜸

<!DOCTYPE html>
<html lang="ko"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width,initial-scale=1,user-scalable=no" name="viewport"/><meta content="telephone=no" name="format-detection"/><meta content="좋아하는 음악은 물론, 좋아할 음악까지 들려주는 취향 저격 뮤직 서비스" name="description"/><meta content="app-id=1338631589" name="apple-itunes-app"/><meta content="website" property="og:type"/><meta content="https://music-phinf.pstatic.net/20201019_118/16031004163365Jexk_JPEG/about.jpg" property="og:image"/><meta content="NAVER VIBE(바이브)" property="og:title"/><meta content="#나만을 위한 음악이 듣고 싶을 땐 #내돈내듣 VIBE." property="og:description"/><meta content="https://vibe.naver.com" property="og:url"/><meta content="summary_large_image" name="twitter:card"/><meta content="https://music-phinf.pstatic.net/20201019_118/16031004163365Jexk_JPEG/about.jpg" name="twitter:image"/><meta content="NAVER VIBE(바이브)" name="twitter:title"/><meta content="NAVER VIBE(바이브)" name=

In [6]:
# conda install selenium  -> 사람손 대신 클릭해주는거(?)
# 크롬 드라이버 다운(ChromeDriver) -> 자기 크롬 버전에 맞는걸로 받아야 함. -> 도움말->크롬 정보에서 확인(92.0.4515.XXX)
# chromedriver.exe 가 나오는데 -> (그냥두면 더러우니 정리하려고) JupyterNotebook폴더에 driver 폴더 생성후 거기에 놓기
from selenium import webdriver
driver = webdriver.Chrome('driver/chromedriver') # 드라이버 위치 줘야 함, .exe는 빼야함
driver.get('https://vibe.naver.com/chart/total')  # 찾을 사이트 주소 넣기 -> 방화벽뜨는거 해제
# -> 그리고 이러면 창이 하나 더 뜨는데 -> 이건 selenium 이 띄운 것
# 그 창에서 다시 우클릭->검사->원하는부분 클릭(여기선 팝업의 창끄기용 X버튼)
# 원하는 부분 나왔으면, 오른쪽 검사부분에 우클릭 -> copy -> copy Xpath

In [7]:
xpath = """//*[@id="app"]/div[2]/div/div/a[2]"""
# driver.find_element_by_xpath(xpath)   # -> 이러면 크롬드라이버가 찾아감
driver.find_element_by_xpath(xpath).click()   # -> 찾아간 다음 .click()으로 클릭시킴 ->그러면 클릭해서 (팝업)꺼짐 
                        # ->(새로 열린 크롬창에서 확인해보면 켜져있던 팝업창이 꺼진걸 확인 가능) 

In [10]:
# html_music = requests.get(url).text   #  -> 아래로 변경
html_music = driver.page_source
soup_music = BeautifulSoup(html_music, 'lxml')

In [11]:
titles = soup_music.select('span.inner_cell a.link_text')
print(len(titles))  # 이제 잘 나옴

100


In [12]:
print(titles[:2])

[<a class="link_text" href="/track/50071944" title="낙하 (with 아이유)">낙하 (with 아이유)</a>, <a class="link_text" href="/track/48124900" title="신호등">신호등</a>]


In [13]:
music_titles = [title.get_text() for title in titles]
print(music_titles[:10])

['낙하 (with 아이유)', '신호등', 'Next Level', 'Permission to Dance', 'Weekend', 'Stay', 'Butter', '바라만 본다', 'Queendom', 'DUMB DUMB']


In [14]:
# 이번엔 아티스트 뽑아봅시다
# class=artist 라는  td 안에 있는 <a>text
artists = soup_music.select('td.artist a.link_artists span.text')  # -> 이러면 안됨
print(len(artists))

0


In [30]:
srtists = soup_music.select('td.artist').get('title')  #  이것도 안됨
print(len(artists))

AttributeError: ResultSet object has no attribute 'get'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [15]:
artists = soup_music.select('td.artist')  # 이렇게 해야 함
print(len(artists))

100


In [16]:
print(artists[:2])  #  td안에있는 span까지 다 갖고 나옴. -> 여기서 원하는건 title 속성만 갖고 오는 것

[<td class="artist" title="AKMU(악뮤)"><span class="inner"><span><span><a class="link_artist" href="/artist/243100"><span class="text">AKMU(악뮤)</span></a><!-- --></span></span></span></td>, <td class="artist" title="이무진"><span class="inner"><span><span><a class="link_artist" href="/artist/2389293"><span class="text">이무진</span></a><!-- --></span></span></span></td>]


In [17]:
artists = [artist['title'] for artist in artists]  #  여기서 artists를 꺼내서 -> title 속성만 하나씩 꺼내 오는 것

In [18]:
print(len(artists))

100


In [19]:
print(artists[:10])

['AKMU(악뮤)', '이무진', 'aespa', '방탄소년단', '태연 (TAEYEON)', 'The Kid LAROI, Justin Bieber', '방탄소년단', 'MSG워너비(M.O.M)', 'Red Velvet (레드벨벳)', '전소미']


In [20]:
driver.close()

In [22]:
# 지금까지 있던걸 함수로 만들어 봄  .  위에서부터 하나씩 조립해보자
def get_vibe_top100(url):
    driver = webdriver.Chrome('driver/chromedriver')
    driver.get(url)
    xpath = """//*[@id="app"]/div[2]/div/div/a[2]"""
    driver.find_element_by_xpath(xpath).click()
    html_music = driver.page_source
    soup_music = BeautifulSoup(html_music, 'lxml')
    
    titles = soup_music.select('span.inner_cell a.link_text')
    music_titles = [title.get_text() for title in titles]
    
    artists = soup_music.select('td.artist') # 아티스트부분 가져오고
    music_artists = [artist['title'] for artist in artists]  # 변수 통일함 music_
    
    for k in range(len(music_titles)):
        print("{0}: {1} / {2}\n".format(k+1, \
                                       music_titles[k], music_artists[k]))
    
    driver.close()

In [23]:
get_vibe_top100('https://vibe.naver.com/chart/total')

1: 낙하 (with 아이유) / AKMU(악뮤)

2: 신호등 / 이무진

3: Next Level / aespa

4: Permission to Dance / 방탄소년단

5: Weekend / 태연 (TAEYEON)

6: Stay / The Kid LAROI, Justin Bieber

7: Butter / 방탄소년단

8: 바라만 본다 / MSG워너비(M.O.M)

9: Queendom / Red Velvet (레드벨벳)

10: DUMB DUMB / 전소미

11: Bad Habits / Ed Sheeran

12: Dun Dun Dance / 오마이걸(OH MY GIRL)

13: 좋아좋아 / 조정석

14: 헤픈 우연 / 헤이즈 (Heize)

15: Alcohol-Free / TWICE(트와이스)

16: Peaches (Feat. Daniel Caesar, Giveon) / Justin Bieber

17: 라일락 / 아이유(IU)

18: 비와 당신 / 이무진

19: Dynamite / 방탄소년단

20: 롤린 (Rollin') / 브레이브걸스(Brave Girls)

21: ASAP / STAYC(스테이씨)

22: OHAYO MY NIGHT / 디핵(D-Hack), PATEKO(파테코)

23: Celebrity / 아이유(IU)

24: 여전히 아름다운지 / 세븐틴

25: 안녕 (Hello) / 조이 (JOY)

26: Kiss Me More (Feat. SZA) / Doja Cat

27: 비 오는 날 듣기 좋은 노래 (Feat. Colde) / 에픽하이 (EPIK HIGH)

28: 치맛바람 (Chi Mat Ba Ram) / 브레이브걸스(Brave Girls)

29: 나를 아는 사람 / MSG워너비(정상동기)

30: 슈퍼스타 / 미도와 파라솔

31: 추적이는 여름 비가 되어 / 장범준

32: 하루만 더 / 빅마마

33: 가을 우체국 앞에서 / 김대명

34: 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 / AKMU(악